In [25]:
import os
import numpy as np
import pandas as pd

from sys import path
path.append("../analysis/utils/")

from utils import get_datasets

In [26]:
DATASETS = ["reut", "acm"]

DATA_SOURCE = "/home/welton/data"

ORACLE_DIR = f"{DATA_SOURCE}/oracle/upper_bound"

In [27]:
pd_datasets = get_datasets(DATASETS, path=f"{DATA_SOURCE}/pd_datasets/__dset__.csv", sep=';')

In [28]:
# For each dataset.
for dset in DATASETS:
    df = pd_datasets[dset]
    labels = df[["label", "fold_id"]]
    try:
        preds = df.drop(columns=["label", "fold_id", "rep_bert_bkp", "bert"])
    except:
        preds = df.drop(columns=["label", "fold_id", "bert"])
    # For each classifier.
    for clf in preds.columns:
        # For each fold.
        for fold in np.arange(10):
            # Making train upper bound.
            fold_preds = preds[labels.fold_id != fold]
            fold_labels = labels[labels.fold_id != fold]
            train_upper = np.zeros(fold_preds.shape[0])
            # Set Hit/True (value 1) on documents right classifieds.
            train_upper[fold_preds[clf].values == fold_labels.label.values] = 1
            output_dir = f"{ORACLE_DIR}/{dset}/{clf}/{fold}/"
            os.makedirs(output_dir, exist_ok=True)
            output = f"{output_dir}/train"
            # Save the upper_ground.
            np.savez(output, y=train_upper)

            # Making test upper bound.
            fold_preds = preds[labels.fold_id == fold]
            fold_labels = labels[labels.fold_id == fold]
            test_upper = np.zeros(fold_preds.shape[0])
            # Set Hit/True (value 1) on documents right classifieds.
            test_upper[fold_preds[clf].values == fold_labels.label.values] = 1
            output = f"{output_dir}/test"
            # Save the upper_ground.
            np.savez(output, y=test_upper)

In [29]:
(np.load("/home/welton/data/oracle/upper_bound/20ng/kfr/0/train.npz")['y'].shape,
np.load("/home/welton/data/oracle/upper_bound/20ng/kfr/0/test.npz")['y'].shape)

((16954,), (1892,))

In [30]:
m = np.array([[1,1,1],[1,1,1],[1,1,1]])
m

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])

In [31]:
p = np.array([1,0,1])
p

array([1, 0, 1])

In [32]:
m * p[:, None]

array([[1, 1, 1],
       [0, 0, 0],
       [1, 1, 1]])